<a href="https://colab.research.google.com/github/JBtallgrass/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-20 02:16:38--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  3.94MB/s    in 0.2s    

2022-10-20 02:16:39 (3.94 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



### Load Amazon Data into Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    1774101|R3PMQXEVYR4ZT1|B00DVKOYBM|     263046246|         8 Ball Pool|     Mobile_Apps|          3|            0|          0|   N|                Y|                 Fun|It is a fun game ...|2015-08-31 00:00:00|
|         US|   13679234|R1I1K3EO98EMUV|B00R1IA4RS|     128868854|Christmas Cookie ...| 

### Create DataFrames to match tables

In [8]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df.select(["review_id","customer_id","product_id","product_parent","product_title","review_date","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
review_df.show()

review_df2 = review_df.dropDuplicates(["review_id"])
print("Distinct count: ",review_df2.count())
print("Distinct count: ",review_df.count())

+--------------+-----------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+
|     review_id|customer_id|product_id|product_parent|       product_title|        review_date|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+
|R3PMQXEVYR4ZT1|    1774101|B00DVKOYBM|     263046246|         8 Ball Pool|2015-08-31 00:00:00|          3|            0|          0|   N|                Y|
|R1I1K3EO98EMUV|   13679234|B00R1IA4RS|     128868854|Christmas Cookie ...|2015-08-31 00:00:00|          5|            0|          0|   N|                Y|
|R3K23URSI2MY42|   15056632|B01069LGHK|     955481764|Relax Melodies Pr...|2015-08-31 00:00:00|          5|            4|          4|   N|                Y|
| RH2P770T10X38|   52477711|B00X64ZKM0|     455029830|  St

In [9]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = review_df.groupby("customer_id").agg({"customer_id": 'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   44552694|             7|
|   27457155|             6|
|   35842367|             9|
|   16848166|             5|
|   13313689|            10|
|    3628854|             1|
|   43626894|             1|
|    4658983|             8|
|   16457323|             3|
|   30024726|             5|
|    3636174|             1|
|     582750|             1|
|      78113|             1|
|    2246607|             5|
|   26264517|             1|
|    7439900|             1|
|    2361059|             2|
|   22965400|             2|
|   32777038|             1|
|   29019526|             2|
+-----------+--------------+
only showing top 20 rows



In [10]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()
products_df = review_df.select(["product_id","product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004752CB4|Brigham's Airport...|
|B004981S0K|           Generic K|
|B004AFQAUA|             Napster|
|B004ALJIOE|     Fast Reboot Pro|
|B004ALVL6W|    Bubble Defense 2|
|B004AMDC86|Enigma - Cryptograms|
|B004AMFUYK|   Vegas Pool Sharks|
|B004AMIYJS| 3D Spider Walk Free|
|B004AMLATE|       Tank Ace 1944|
|B004AMRGLA|             Gragger|
|B004ANC00Q|         Car Locator|
|B004ANE2WU|         Par 72 Golf|
|B004AZH4C8|        Enjoy Sudoku|
|B004AZSY4K|Overkill: Space S...|
|B004BH1B9M|          Lights Out|
|B004BN3YQE|       Word Collapse|
|B004CN7Y4G|           Thesaurus|
|B004DJZBN0|Total Recall Call...|
|B004DKEGQC|            Kool App|
|B004DKSUXC|    Spider Solitaire|
+----------+--------------------+
only showing top 20 rows



In [11]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = review_df.select(["review_id","customer_id", "product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()
review_id_df.count()

review_id_df = review_id_df.distinct()
review_id_df.count()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3PMQXEVYR4ZT1|    1774101|B00DVKOYBM|     263046246| 2015-08-31|
|R1I1K3EO98EMUV|   13679234|B00R1IA4RS|     128868854| 2015-08-31|
|R3K23URSI2MY42|   15056632|B01069LGHK|     955481764| 2015-08-31|
| RH2P770T10X38|   52477711|B00X64ZKM0|     455029830| 2015-08-31|
| RV9YBAW8CWFYN|   15021042|B00KIZO238|     685358159| 2015-08-31|
| R19627K7G1G6X|    5181462|B00H9FE0FU|     158085302| 2015-08-31|
|R22858XECEF7YQ|   19360265|B0064X7FVE|     140639925| 2015-08-31|
|R3KQOY7MDZOK0U|   28928572|B00N28818A|     838378762| 2015-08-31|
|R31BKIN02QFDHS|   26842555|B00HGMXSGI|     141073643| 2015-08-31|
|R3FW62P5EUDUUV|   25535583|B0119R2DB6|     498391460| 2015-08-31|
| RBD8ROQJQO68E|    3343825|B00O0OR65E|     156086007| 2015-08-31|
|R1S04R7IT12FKH|   49181739|B00YCL9R0W|     629058532| 2015-08

5033376

In [12]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = review_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3PMQXEVYR4ZT1|          3|            0|          0|   N|                Y|
|R1I1K3EO98EMUV|          5|            0|          0|   N|                Y|
|R3K23URSI2MY42|          5|            4|          4|   N|                Y|
| RH2P770T10X38|          5|            0|          0|   N|                Y|
| RV9YBAW8CWFYN|          1|            0|          0|   N|                Y|
| R19627K7G1G6X|          5|            0|          0|   N|                Y|
|R22858XECEF7YQ|          3|            0|          0|   N|                N|
|R3KQOY7MDZOK0U|          1|            0|          0|   N|                Y|
|R31BKIN02QFDHS|          4|            0|          0|   N|                Y|
|R3FW62P5EUDUUV|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.c14se7ofvwud.us-east-2.rds.amazonaws.com:5432/Amazon_Vine_Analysis"
config = {"user":"postgres", 
          "password": "kc101010", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)